<a href="https://colab.research.google.com/github/Mollylst/resnet18/blob/main/custom_dataset_trainandtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

In [ ]:
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((132, 132)),  
    transforms.RandomCrop((128, 128)),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

transform_test = transforms.Compose([
     transforms.ToPILImage(),
     transforms.Resize((132, 132)),  
     transforms.RandomCrop((128, 128)), 
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

class ApplyTransform(Dataset):
    """
    Apply transformations to a Dataset

    Arguments:
        dataset (Dataset): A Dataset that returns (sample, target)
        transform (callable, optional): A function/transform to be applied on the sample
        target_transform (callable, optional): A function/transform to be applied on the target

    """
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform
        # yes, you don't need these 2 lines below :(
        if transform is None and target_transform is None:
            print("Am I a joke to you? :)")

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

    def __len__(self):
        return len(self.dataset)

In [ ]:
class DeepfakeDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
in_channel = 3
num_classes = 2
batch_size = 32

# Load Data
dataset = DeepfakeDataset(
    csv_file="/content/drive/MyDrive/files_and_labels.csv",
    root_dir="/content/drive/MyDrive/real_and_fake_face_together",
    transform=transforms.ToTensor(),
)

#split

train_set, test_set = torch.utils.data.random_split(dataset, [1632, 409])
#lengths = [int(len(dataset)*0.8), int(len(dataset)*0.2)]
#lengths = [1632, 409]
#subsetA, subsetB = torch.utils.data.random_split(dataset, lengths)

trainset = ApplyTransform(train_set,transform=transform_train)
testset = ApplyTransform(test_set,transform=transform_test)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

cuda


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.reshape(out.shape[0],-1)
        #out = F.avg_pool2d(out, 4)
        #out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#define ResNet18
net = ResNet18().to(device)

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=0, last_epoch=-1, verbose=False)

In [ ]:
import time
num_epoch=15

def train_and_test():
    history = []
    best_acc = 0.0
    best_epoch = 0
    
    for epoch in range(0,num_epoch):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, num_epoch))

        net.train()
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0

        #train
        for i, (inputs, labels) in enumerate(trainloader):

            inputs, labels = inputs.to(device), labels.to(device)
 
            #the gradient is increasing, so make it zero
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
 
            train_loss += loss.item() * inputs.size(0)
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            train_acc += acc.item() * inputs.size(0)

        #test
        net.eval()
        with torch.no_grad():

            for j, (inputs, labels) in enumerate(testloader):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
 
                loss = criterion(outputs, labels)
 
                test_loss += loss.item() * inputs.size(0)
 
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
 
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
 
                test_acc += acc.item() * inputs.size(0)
 
        avg_train_loss = train_loss/ len(trainset)
        avg_train_acc = train_acc/ len(trainset)
 
        avg_test_loss = test_loss/len(testset)
        avg_test_acc = test_acc/len(testset)

        #record the accuracy and loss in each epoch
        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
 
        if best_acc < avg_test_acc:
            best_acc = avg_test_acc
            best_epoch = epoch + 1

        scheduler.step()  

        epoch_end = time.time()
        
        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tTest: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for test : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
          
    return history
    
#start training and testing
history = train_and_test()

#make the plots
history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1.1)
#plt.savefig(dataset+'_loss_curve.png')
plt.show()
 
plt.plot(history[:, 2:4])
plt.legend(['Training Accuracy', 'Test Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1.1)
#plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

Epoch: 1/30
Epoch: 001, Training: Loss: 0.9709, Accuracy: 48.0392%, 
		Test: Loss: 0.7566, Accuracy: 46.4548%, Time: 335.2110s
Best Accuracy for test : 0.4645 at epoch 001
Epoch: 2/30
Epoch: 002, Training: Loss: 0.8035, Accuracy: 51.2868%, 
		Test: Loss: 0.9435, Accuracy: 46.4548%, Time: 158.4939s
Best Accuracy for test : 0.4645 at epoch 001
Epoch: 3/30
Epoch: 003, Training: Loss: 0.7387, Accuracy: 55.8211%, 
		Test: Loss: 0.7203, Accuracy: 53.0562%, Time: 156.7049s
Best Accuracy for test : 0.5306 at epoch 003
Epoch: 4/30
Epoch: 004, Training: Loss: 0.6760, Accuracy: 59.5588%, 
		Test: Loss: 0.7669, Accuracy: 50.6112%, Time: 158.9065s
Best Accuracy for test : 0.5306 at epoch 003
Epoch: 5/30
Epoch: 005, Training: Loss: 0.6500, Accuracy: 60.9069%, 
		Test: Loss: 0.6952, Accuracy: 56.7237%, Time: 157.8010s
Best Accuracy for test : 0.5672 at epoch 005
Epoch: 6/30
Epoch: 006, Training: Loss: 0.6587, Accuracy: 60.9069%, 
		Test: Loss: 0.7509, Accuracy: 53.3007%, Time: 159.1344s
Best Accuracy

NameError: ignored

In [ ]:
PATH = './real_fake.pth'
torch.save(net.state_dict(), PATH)